In [2]:
import warnings
warnings.filterwarnings("ignore")
import pyart
import numpy as np
from pyart.retrieve import get_freq_band
from matplotlib import pyplot as plt
import glob
import os
import time
import cow1_funcs as cf


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead


In [3]:
def get_file_list(directory):
    files = []
    for root, dirs, filenames in os.walk(directory):
        for filename in filenames:
            files.append(os.path.join(root, filename))
    files.sort()
    return files

basedir = "/depot/dawson29/data/Projects/PERiLS/obsdata/2022/Illinois_Mobile_Radar/IOP2/"
cow_dir = os.path.join(basedir, "COW1/merged/")

file_list = get_file_list(cow_dir)
num_files = len(file_list)

print("Number of files:", num_files)

Number of files: 319


In [4]:
radar = pyart.io.read(file_list[104])
refl_field = "DBZHCC_F"
vel_field = "VEL_F"
ncp_field = "NCP"
refl_thresh = 5 
ncp_thresh = None
rho_field = "RHOHV"
rho_thresh = 0.45
# radar = cf.filter_data(radar, refl_field,
#                     refl_thresh=refl_thresh,
#                     vel_field=vel_field,
#                     ncp_field=ncp_field,
#                     ncp_thresh=ncp_thresh,
#                     rho_field=rho_field,
#                     rho_thresh=rho_thresh,
#                     dealias_method='region',
#            )

In [5]:
radar.fields.keys()

dict_keys(['NCP', 'TRIP_FLA', 'SNRHC', 'SNRVC', 'DBMHC', 'DBMVC', 'DBZHC', 'DBZHC_F', 'DBZVC', 'DBZVC_F', 'VEL', 'VEL_F', 'VS', 'VS_F', 'VL', 'VL_F', 'WIDTH', 'ZDRM', 'RHOHV', 'PHIDP', 'KDP', 'DBZHCC', 'DBZHCC_F', 'ZDRC'])

In [6]:
def linspace_range(start, stop, step):
    num = int((stop - start) / step + 1)
    return num
print(f'KGWX (v, h): {linspace_range(0, 10e3, 250)}, {linspace_range(0, 200*1e3, 500)}')
print(f'COW1 (v, h): {linspace_range(0, 10e3, 250)}, {linspace_range(0, 88444.0*2, 500)}')

KGWX (v, h): 41, 401
COW1 (v, h): 41, 354


In [7]:
np.ceil(radar.range['data'].max())

88444.0

In [8]:
radar.instrument_parameters['unambiguous_range']['data'].max()

89937.734

In [9]:
def _rename_moms(radar):
    '''Rename fields'''
    radar.fields['REF'] = radar.fields.pop('DBZHCC_F')
    radar.fields['VEL'] = radar.fields.pop('VEL_F')
    radar.fields['ZDR'] = radar.fields.pop('ZDRC')
    radar.fields['RHO'] = radar.fields.pop('RHOHV')
    
    return radar

In [ ]:
radar = _rename_moms(radar)
max_rng = 88000
grid = pyart.map.grid_from_radars(radar, (41, 354, 354),
                                  ((0., 10e3), (-max_rng, max_rng), (-max_rng, max_rng)),
                                  weighting_function='Barnes2',
                                  fields=['REF'])

In [1]:
linspace_range(0, 88.25*1e3*2, 500)

NameError: name 'linspace_range' is not defined

In [ ]:
np.diff(grid.x['data'])

In [14]:
grid.x

{'standard_name': 'projection_x_coordinate',
 'long_name': 'X distance on the projection plane from the origin',
 'axis': 'X',
 'units': 'm',
 'data': array([-88444.        , -87942.90084986, -87441.80169972, -86940.70254958,
        -86439.60339943, -85938.50424929, -85437.40509915, -84936.30594901,
        -84435.20679887, -83934.10764873, -83433.00849858, -82931.90934844,
        -82430.8101983 , -81929.71104816, -81428.61189802, -80927.51274788,
        -80426.41359773, -79925.31444759, -79424.21529745, -78923.11614731,
        -78422.01699717, -77920.91784703, -77419.81869688, -76918.71954674,
        -76417.6203966 , -75916.52124646, -75415.42209632, -74914.32294618,
        -74413.22379603, -73912.12464589, -73411.02549575, -72909.92634561,
        -72408.82719547, -71907.72804533, -71406.62889518, -70905.52974504,
        -70404.4305949 , -69903.33144476, -69402.23229462, -68901.13314448,
        -68400.03399433, -67898.93484419, -67397.83569405, -66896.73654391,
        -66395

In [17]:
outdir = "/depot/dawson29/data/Projects/PERiLS/obsdata/2022/GRID/IOP2/"
grid_file_path = os.path.join(outdir, "COW1")

In [16]:
grid_file_path

'/depot/dawson29/data/Projects/PERiLS/obsdata/2022/GRID/IOP2/COW1'

In [23]:
os.makedirs(grid_file_path, exist_ok=True)

In [26]:
ls $outdir/

COW1/  KGWX/


In [4]:
import subprocess

sbatch_command = "sbatch sbatch_parallel_gridding.sbatch"
output = subprocess.check_output(sbatch_command, shell=True)
job_id = output.decode('utf-8').strip().split()[-1]

print("Submitted job with ID:", job_id)

Submitted job with ID: 25688368


In [15]:
!squeue -A dawson29

JOBID        USER      ACCOUNT      NAME             NODES   CPUS  TIME_LIMIT ST TIME
25630030     syed44    dawson29     OnDemand/Noteboo     1     16  5-00:00:00  R 2-05:52:57
25682071     jiang703  dawson29     OnDemand/Noteboo     1     32    12:00:00  R 6:50:19


In [40]:
!tail parallel_gridding.out

Saving in: /depot/dawson29/data/Projects/PERiLS/obsdata/2022/GRID/IOP2/COW1 as cfrad.20220330_172523.534_to_20220330_172645.975_COW1high_SUR.nc

Reading file: cfrad.20220330_223009.756_to_20220330_223139.835_COW1high_SUR.nc
Deleting Radar Object: cfrad.20220330_223009.756_to_20220330_223139.835_COW1high_SUR.nc
Saving in: /depot/dawson29/data/Projects/PERiLS/obsdata/2022/GRID/IOP2/COW1 as cfrad.20220330_223009.756_to_20220330_223139.835_COW1high_SUR.nc

Reading file: cfrad.20220331_021013.827_to_20220331_021143.952_COW1high_SUR.nc
Deleting Radar Object: cfrad.20220331_021013.827_to_20220331_021143.952_COW1high_SUR.nc
Saving in: /depot/dawson29/data/Projects/PERiLS/obsdata/2022/GRID/IOP2/COW1 as cfrad.20220331_021013.827_to_20220331_021143.952_COW1high_SUR.nc



In [42]:
!ls -lth /depot/dawson29/data/Projects/PERiLS/obsdata/2022/GRID/IOP2/COW1/ | wc

    320    2873   37333


In [43]:
!scontrol show job $job_id

slurm_load_jobs error: Invalid job id specified


## UAH Gridding

In [39]:
basedir = "/depot/dawson29/data/Projects/PERiLS/obsdata/2022/UAH_Mobile_Radar/"
files = sorted(glob.glob(os.path.join(basedir, "RAW*2022033*nc")))
print(f'Number of files: {len(files)}')

Number of files: 242


In [88]:
# function to dealiase the Doppler velocity
def dealiase(radar, vel_name='velocity', gatefilter=None, method="unwrap"):
    '''
    Dealias Doppler velocities using Py-ART.
    method : str
        Method to use for the dealiasing. Can be 'unwrap' or 'region'.
    '''
    # Create a GateFilter if one was not provided
    if gatefilter is None:
        gatefilter = pyart.correct.GateFilter(radar)
    # Dealias Doppler velocities using the selected method
    if method == "unwrap":
        corr_vel = pyart.correct.dealias_unwrap_phase(radar, vel_field=vel_name,
                                                      keep_original=False, gatefilter=gatefilter)
    elif method == "region":
        corr_vel = pyart.correct.dealias_region_based(radar, vel_field=vel_name,
                                                      keep_original=False, gatefilter=gatefilter)
    # Add the dealiased Doppler velocities to the radar object
    radar.add_field(vel_name, corr_vel, replace_existing=True)
    return radar

In [16]:
def _drop_fields(radar):
    fields_to_drop = ["total_power", "spectrum_width", "differential_reflectivity",
                      'specific_differential_phase', 'differential_phase', 'normalized_coherent_power']
    for field in fields_to_drop:
        if field in radar.fields:
            del radar.fields[field]
    return radar

In [19]:
def _rename_moms(radar):
    '''Rename fields'''
    radar.fields['REF'] = radar.fields.pop('reflectivity')
    radar.fields['VEL'] = radar.fields.pop('velocity')
    radar.fields['ZDR'] = radar.fields.pop('c_zdr')
    radar.fields['RHO'] = radar.fields.pop('cross_correlation_ratio')

    return radar

In [22]:
radar = pyart.io.read(files[100])
radar = _drop_fields(radar)
radar = _rename_moms(radar)

In [23]:
radar.fields.keys()

dict_keys(['REF', 'VEL', 'ZDR', 'RHO'])

In [27]:
radar.range['data'].max()

99750.0

In [28]:
def linspace_range(start, stop, step):
    num = ((stop - start) / step + 1)
    return num

In [31]:
max_rng = 99750.0
linspace_range(-max_rng, max_rng, 500)

400.0

In [32]:
max_rng = 99750.0
grid = pyart.map.grid_from_radars(radar, (41, 400, 400),
                                  ((0., 10e3), (-max_rng, max_rng),
                                   (-max_rng, max_rng)),
                                  weighting_function='Barnes2',
                                  fields=['REF'])

In [37]:
grid.x['data']/1e3

array([-99.75, -99.25, -98.75, -98.25, -97.75, -97.25, -96.75, -96.25,
       -95.75, -95.25, -94.75, -94.25, -93.75, -93.25, -92.75, -92.25,
       -91.75, -91.25, -90.75, -90.25, -89.75, -89.25, -88.75, -88.25,
       -87.75, -87.25, -86.75, -86.25, -85.75, -85.25, -84.75, -84.25,
       -83.75, -83.25, -82.75, -82.25, -81.75, -81.25, -80.75, -80.25,
       -79.75, -79.25, -78.75, -78.25, -77.75, -77.25, -76.75, -76.25,
       -75.75, -75.25, -74.75, -74.25, -73.75, -73.25, -72.75, -72.25,
       -71.75, -71.25, -70.75, -70.25, -69.75, -69.25, -68.75, -68.25,
       -67.75, -67.25, -66.75, -66.25, -65.75, -65.25, -64.75, -64.25,
       -63.75, -63.25, -62.75, -62.25, -61.75, -61.25, -60.75, -60.25,
       -59.75, -59.25, -58.75, -58.25, -57.75, -57.25, -56.75, -56.25,
       -55.75, -55.25, -54.75, -54.25, -53.75, -53.25, -52.75, -52.25,
       -51.75, -51.25, -50.75, -50.25, -49.75, -49.25, -48.75, -48.25,
       -47.75, -47.25, -46.75, -46.25, -45.75, -45.25, -44.75, -44.25,
      

In [299]:
import subprocess

sbatch_command = "sbatch sbatch_parallel_gridding.sbatch"
output = subprocess.check_output(sbatch_command, shell=True)
job_id = output.decode('utf-8').strip().split()[-1]

print("Submitted job with ID:", job_id)

Submitted job with ID: 25720272


In [326]:
!squeue -A dawson29

JOBID        USER      ACCOUNT      NAME             NODES   CPUS  TIME_LIMIT ST TIME
25630030     syed44    dawson29     OnDemand/Noteboo     1     16  5-00:00:00  R 3-09:23:33
25706197     kaxon     dawson29     OnDemand/Desktop     1      4  2-12:00:00  R 7:19:24
25720138     jiang703  dawson29     R50r15kmCdFreeC1    12    400  3-00:00:00  R 1:06:24


In [322]:
!tail parallel_gridding.out

## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119

Number of files in DOW7 directory: 221


In [325]:
!ls -lth /depot/dawson29/data/Projects/PERiLS/obsdata/2022/GRID/IOP2/DOW7 | wc

    222    1991   25868


In [306]:
!ls -lth /depot/dawson29/data/Projects/PERiLS/obsdata/2022/GRID/IOP2/DOW8/ | wc

    368    3305   41482


In [114]:
# !scancel $job_id